In [1]:
import os
import time
from time import sleep
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
load_dotenv()

True

In [3]:
# setup coinmarketcap api
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'20',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': os.environ["X-CMC_PRO_API_KEY"],
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  # print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [4]:
symbol=[]
market_caps=[]
date_added=[]
symbol_name=[]
# new_dataframe = pd.DataFrame(ticker, columns=cols)
for crypto_data in data['data']:
    ticker=crypto_data['symbol']
    market_cap=crypto_data['quote']['USD']['market_cap']
    name=crypto_data['slug']
    genesis=crypto_data['date_added']
    symbol.append(ticker)
    market_caps.append(market_cap)
    symbol_name.append(name)
    date_added.append(genesis)
print(symbol_name)

['bitcoin', 'ethereum', 'tether', 'polkadot-new', 'xrp', 'cardano', 'litecoin', 'bitcoin-cash', 'chainlink', 'stellar', 'binance-coin', 'usd-coin', 'wrapped-bitcoin', 'bitcoin-sv', 'monero', 'eos', 'uniswap', 'tezos', 'aave', 'tron']


In [5]:
cols=['Symbol', 'Market Cap']
top_20_crypto = pd.DataFrame({"Symbol":symbol, "Symbol name": symbol_name, "Market Cap": market_caps, "Date added": date_added})
top_20_crypto["Symbol name"]=top_20_crypto["Symbol name"].str.replace("-","")
top_20_crypto["Symbol name"].replace({"polkadotnew": "polkadot"}, inplace=True)

In [6]:
# pull cryptocurrency news for each coin using gnews api
start_date="2017-01-16"
end_date="2021-01-16"
gnews_api = os.environ["gnews_api"]
gnews_data = []

for articles in top_20_crypto['Symbol name']:  
    gnews_url =f"https://gnews.io/api/v4/search?q={articles}&in=cryptocurrency&from=start_date&to=end_date&lang=en&token={gnews_api}"
    response = requests.get(gnews_url)
    gnews_data.append(response.json())
    print(articles)
    time.sleep(4)

bitcoin
ethereum
tether
polkadot
xrp
cardano
litecoin
bitcoincash
chainlink
stellar
binancecoin
usdcoin
wrappedbitcoin
bitcoinsv
monero
eos
uniswap
tezos
aave
tron


In [7]:
articles=[]
for article in gnews_data:
    articles.append(article)

In [ ]:
newsapi= NewsApiClient(api_key=os.environ["NEWS_API_KEY"])
print(type(newsapi))

In [ ]:
# pull cryptocurrency news for each coin using gnews api
start_date="2020-12-18"
end_date="2021-01-16"
newsapi_data = []
for symbols in top_20_crypto['Symbol name']:  
    top_headlines = newsapi.get_top_headlines(q=symbols,
                                          category='business',
                                          language='en',
                                          country='us')
    newsapi_data.append(top_headlines)

In [ ]:
sources = newsapi.get_sources()
sources

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Create the sentiment scores DataFrame
crypto_senti = []
for article in crypto_articles['articles']:
    try:
        text = article['content']
        senti = analyzer.polarity_scores(text)
        compound = senti['compound']
        pos = senti['pos']
        neu = senti['neu']
        neg = senti['neg']
        crypto_senti.append({
            'Text': text,
            'Compound': compound,
            'Positive': pos,
            'Neutral': neu,
            'Negative': neg
        })
    except AttributeError:
        pass
# DataFrame
crypto_df = pd.DataFrame(crypto_senti)
crypto_df.head()

In [ ]:
# Function for putting articles into dataframe
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article[“title”]
            description = article[“description”]
            text = article[“content”]
            date = article[“publishedAt”][:10]
            articles.append({
                “title”: title,
                “description”: description,
                “text”: text,
                “date”: date,
                “language”: language
            })
        except AttributeError:
            pass
    return pd.DataFrame(articles)